In [11]:
import os
import logging

# Suppress PyTorch warnings
os.environ['TORCH_CPP_LOG_LEVEL'] = 'WARNING'

# Configure logging to show only warnings and errors
logging.basicConfig(level=logging.WARNING)


In [13]:
if __name__ == "__main__":
    print("All libraries imported successfully!")


All libraries imported successfully!


In [15]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="torch")


In [2]:
# Verify required libraries
import llama_index
import langchain
import streamlit
import chromadb
import PyPDF2
from sentence_transformers import SentenceTransformer

print("All libraries imported successfully!")



All libraries imported successfully!


In [7]:
import PyPDF2

# Function to extract text from a PDF
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
    return text

# Define the paths to your PDFs
pdf_paths = {
    'Alphabet': r"C:\Users\meghn\OneDrive\Documents\goog-10-k-2023 (1).pdf",
    'Tesla': r"C:\Users\meghn\OneDrive\Documents\tsla-20231231-gen.pdf",
    'Uber': r"C:\Users\meghn\OneDrive\Documents\uber-10-k-2023.pdf"
}

# Extract text from the PDFs
pdf_texts = {company: extract_text_from_pdf(path) for company, path in pdf_paths.items()}

# Print a snippet of the text for verification
for company, text in pdf_texts.items():
    print(f"{company} PDF text snippet: {text[:500]}...")  # Adjust the snippet size as needed


Alphabet PDF text snippet: UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
___________________________________________
FORM 10-K  
___________________________________________
(Mark One)
☒ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the fiscal year ended December 31, 2023  
OR
☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the transition period from              to             .
Commission file number: 001-3...
Tesla PDF text snippet: UNITED	STATES
SECURITIES	AND	EXCHANGE	COMMISSION
Washington,	D.C.	20549
FORM	
10-K
(Mark	One)
x
ANNUAL	REPORT	PURSUANT	TO	SECTION	13	OR	15(d)	OF	THE	SECURITIES	EXCHANGE	ACT	OF	1934
For	the	fiscal	year	ended	
December	31
,	2023
OR
o
TRANSITION	REPORT	PURSUANT	TO	SECTION	13	OR	15(d)	OF	THE	SECURITIES	EXCHANGE	ACT	OF	1934
For	the	transition	period	from	_________	to	_________
Commission	File	Number:	
001-34756
Tesla,	Inc.
(Exact	name	of	registra

In [8]:
import re
import pandas as pd

# Define a function to extract financial data using regex
def extract_financial_data(text):
    # Example regex patterns for revenue and profit (customize based on report structure)
    revenue_pattern = r"Revenue[\s\S]*?(\$[\d,]+(?:\.\d+)?)"
    profit_pattern = r"Profit[\s\S]*?(\$[\d,]+(?:\.\d+)?)"
    
    revenue = re.search(revenue_pattern, text)
    profit = re.search(profit_pattern, text)
    
    # Extracted data, defaulting to None if not found
    revenue = revenue.group(1) if revenue else None
    profit = profit.group(1) if profit else None
    
    return {'Revenue': revenue, 'Profit': profit}

# Apply the extraction function to each company's text
company_data = {}
for company, text in pdf_texts.items():
    company_data[company] = extract_financial_data(text)

# Convert the data into a DataFrame for easier comparison
df = pd.DataFrame(company_data).T
df.to_csv("financial_data_comparison.csv")

print(df)


         Revenue Profit
Alphabet    $100   $2.0
Tesla      $4.68   None
Uber        $1.2   None


In [10]:
import re
import pandas as pd

# Define a function to extract financial data using regex
def extract_financial_data(text):
    # More flexible regex patterns for revenue and profit (customize as needed)
    revenue_pattern = r"(Revenue|Total\s+Revenue|Net\s+Revenue|Sales)[\s\S]*?(\$[\d,]+(?:\.\d+)?)"
    profit_pattern = r"(Profit|Net\s+Income|Operating\s+Income|Gross\s+Profit)[\s\S]*?(\$[\d,]+(?:\.\d+)?)"
    
    revenue = re.search(revenue_pattern, text)
    profit = re.search(profit_pattern, text)
    
    # Extracted data, defaulting to None if not found
    revenue = revenue.group(2) if revenue else None
    profit = profit.group(2) if profit else None
    
    return {'Revenue': revenue, 'Profit': profit}

# Apply the extraction function to each company's text
company_data = {}
for company, text in pdf_texts.items():
    company_data[company] = extract_financial_data(text)

# Convert the data into a DataFrame for easier comparison
df = pd.DataFrame(company_data).T
df.to_csv("financial_data_comparison.csv")

print(df)


         Revenue Profit
Alphabet    $100   $2.0
Tesla     $7,500     $1
Uber         $55   $133


In [13]:
import re
import pandas as pd

# Define a function to extract financial data using regex
def extract_financial_data(text):
    # More flexible regex patterns for revenue and profit (customize as needed)
    revenue_pattern = r"(Revenue|Total\s+Revenue|Net\s+Revenue|Sales)[\s\S]*?(\$[\d,]+(?:\.\d+)?)"
    profit_pattern = r"(Profit|Net\s+Income|Operating\s+Income|Gross\s+Profit)[\s\S]*?(\$[\d,]+(?:\.\d+)?)"
    
    revenue = re.search(revenue_pattern, text)
    profit = re.search(profit_pattern, text)
    
    # Extracted data, defaulting to None if not found
    revenue = revenue.group(2) if revenue else None
    profit = profit.group(2) if profit else None
    
    # Clean the data by removing commas (if any) and converting to a standard format
    if revenue:
        revenue = revenue.replace(',', '').replace('$', '')
    if profit:
        profit = profit.replace(',', '').replace('$', '')
    
    # Convert to float for easier handling in further processing
    if revenue:
        revenue = float(revenue)
    if profit:
        profit = float(profit)
    
    return {'Revenue': revenue, 'Profit': profit}

# Apply the extraction function to each company's text
company_data = {}
for company, text in pdf_texts.items():
    company_data[company] = extract_financial_data(text)

# Convert the data into a DataFrame for easier comparison
df = pd.DataFrame(company_data).T
df.to_csv("financial_data_comparison.csv")

print(df)


          Revenue  Profit
Alphabet    100.0     2.0
Tesla      7500.0     1.0
Uber         55.0   133.0


In [7]:
import pandas as pd
import matplotlib.pyplot as plt

# Sample data
data = {
    "Company": ["Company A", "Company B", "Company C"],
    "Revenue": [100, 150, 200],
    "Profit": [20, 30, 50],
}
df = pd.DataFrame(data)

# Bar plot
df.plot(x="Company", kind="bar", figsize=(10, 6))

# Customize the plot
plt.title("Revenue and Profit Comparison")
plt.ylabel("Amount ($)")
plt.xlabel("Company")
plt.xticks(rotation=45, ha="right")

# Save the plot
plt.tight_layout()
plt.savefig("revenue_profit_comparison.png")
plt.close()


In [15]:
df.to_excel("financial_data_comparison.xlsx", index=True)


In [21]:
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"


In [22]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight embedding model
embeddings = model.encode(["Sample text for embedding"])
print(embeddings)


[[ 2.34674243e-03  2.40223552e-03  1.83480582e-03 -1.54992491e-02
   7.45987371e-02  5.99153303e-02  3.19676772e-02  2.67468151e-02
   2.59960964e-02 -2.96183098e-02  7.61197042e-03  5.58658689e-03
   3.26719023e-02  2.53468510e-02  1.35427415e-02  5.78699000e-02
   1.17850594e-01  6.70695975e-02 -6.91913627e-03 -4.49889638e-02
  -5.51183429e-03  2.83483025e-02  9.90352556e-02 -5.59788868e-02
   3.67851406e-02 -2.57713255e-03  7.66836200e-03  7.71480054e-02
   9.56553444e-02 -8.26762524e-03  5.24235405e-02 -2.45366376e-02
   6.39095008e-02  1.72637124e-02  7.01922327e-02  5.68613000e-02
  -4.50920463e-02  4.26678173e-02 -2.86563262e-02  7.85986185e-02
   3.01986728e-02 -3.73339374e-03 -9.89009906e-03  6.54189289e-02
   9.82317105e-02 -7.66057819e-02 -5.31081669e-02 -3.27406509e-04
   4.13534790e-03  8.49060491e-02 -6.29577413e-02 -3.20573337e-02
  -7.04800561e-02 -3.50319147e-02 -2.32712403e-02 -4.06736806e-02
   1.42755546e-02 -8.09421390e-02  9.88004822e-03 -3.62574644e-02
   3.77748

In [24]:
from sentence_transformers import SentenceTransformer

# Initialize the model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Sample sentences to compare
sentences = ["Sample text for embedding", "Another sample text for comparison"]

# Compute embeddings for the sentences
embeddings = model.encode(sentences)

# Print the embeddings
for i, embedding in enumerate(embeddings):
    print(f"Embedding for sentence {i+1}: {embedding}")


Embedding for sentence 1: [ 2.34676828e-03  2.40221224e-03  1.83472282e-03 -1.54992305e-02
  7.45986775e-02  5.99153526e-02  3.19677219e-02  2.67468262e-02
  2.59961654e-02 -2.96183135e-02  7.61201466e-03  5.58662089e-03
  3.26719172e-02  2.53468733e-02  1.35428030e-02  5.78698516e-02
  1.17850564e-01  6.70695379e-02 -6.91912463e-03 -4.49890345e-02
 -5.51191019e-03  2.83483267e-02  9.90352854e-02 -5.59788235e-02
  3.67851406e-02 -2.57694907e-03  7.66830845e-03  7.71479160e-02
  9.56552699e-02 -8.26757215e-03  5.24235182e-02 -2.45366357e-02
  6.39095381e-02  1.72637608e-02  7.01921433e-02  5.68612777e-02
 -4.50920835e-02  4.26679254e-02 -2.86562406e-02  7.85985515e-02
  3.01986430e-02 -3.73340002e-03 -9.89011955e-03  6.54189661e-02
  9.82317775e-02 -7.66057447e-02 -5.31080775e-02 -3.27340211e-04
  4.13543219e-03  8.49060118e-02 -6.29577115e-02 -3.20572928e-02
 -7.04800189e-02 -3.50318477e-02 -2.32712962e-02 -4.06735837e-02
  1.42755667e-02 -8.09421986e-02  9.88013111e-03 -3.62574905e-02

In [25]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Compute cosine similarity between embeddings
similarity_matrix = cosine_similarity(embeddings)

# Print the similarity matrix
print("Cosine Similarity Matrix:")
print(similarity_matrix)

# If you want to compare just two sentences
similarity_score = cosine_similarity([embeddings[0]], [embeddings[1]])
print(f"Cosine similarity between the first and second sentence: {similarity_score[0][0]}")


Cosine Similarity Matrix:
[[1.0000001  0.47497296]
 [0.47497296 1.0000002 ]]
Cosine similarity between the first and second sentence: 0.4749728739261627


In [32]:
import os
os.environ["OMP_NUM_THREADS"] = "1"


In [33]:
from sklearn.cluster import KMeans
import numpy as np

# Sample sentences and their embeddings
sentences = ["Sample text for embedding", "Another example of text", "More data for analysis", "Different example"]
embeddings = model.encode(sentences)  # Assume you have already created embeddings for the sentences

# Perform KMeans clustering
num_clusters = 2
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
labels = kmeans.fit_predict(embeddings)

# Group sentences by their cluster label
clusters = {i: [] for i in range(num_clusters)}
for sentence, label in zip(sentences, labels):
    clusters[label].append(sentence)

# Print sentences grouped by their cluster labels
for cluster_label, cluster_sentences in clusters.items():
    print(f"Cluster {cluster_label}:")
    for sentence in cluster_sentences:
        print(f"  - {sentence}")
    print()


E:\Anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
E:\Anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


Cluster 0:
  - More data for analysis

Cluster 1:
  - Sample text for embedding
  - Another example of text
  - Different example



In [34]:
sentence_labels = kmeans.labels_
labeled_sentences = list(zip(sentences, sentence_labels))
for sentence, label in labeled_sentences:
    print(f"Cluster {label}: {sentence}")


Cluster 1: Sample text for embedding
Cluster 1: Another example of text
Cluster 0: More data for analysis
Cluster 1: Different example


In [9]:
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# Sample data
embeddings = np.random.rand(10, 5)  # 10 samples, 5 features
kmeans = KMeans(n_clusters=3)  # Example: clustering into 3 clusters
kmeans.fit(embeddings)

# Reduce to 2D using PCA
pca = PCA(n_components=2)
reduced_embeddings = pca.fit_transform(embeddings)

# Scatter plot
plt.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1], c=kmeans.labels_)
plt.colorbar()

# Save the plot
plt.savefig("pca_scatter_plot.png")
plt.close()


E:\Anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [36]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(embeddings, kmeans.labels_, test_size=0.3, random_state=42)

# Train a classifier (Logistic Regression in this case)
clf = LogisticRegression()
clf.fit(X_train, y_train)

# Evaluate the classifier
print(f"Accuracy: {clf.score(X_test, y_test)}")


Accuracy: 1.0


In [43]:
from datasets import Dataset


In [68]:
from sentence_transformers import SentenceTransformer, losses
from datasets import Dataset
from torch.utils.data import DataLoader


In [69]:
# Create dataset from your training data (example)
train_data = [{'sentence': sentence} for sentence in sentences]  # 'sentences' should be your list of sentences

train_dataset = Dataset.from_dict({'sentence': sentences})  # Convert to Dataset

# Create DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)


In [71]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
from datasets import Dataset  # Correct import of the Dataset class

# Sample sentences for training (you can replace this with your own data)
sentences = [
    'This is a sentence.',
    'This is another sentence.',
    'How are you doing today?',
    'I am learning machine learning.',
    'Sentence transformers are great for NLP tasks.'
]

# Create InputExamples from the sentences for training
# Here we use the same sentence pair for simplicity, but you can pair them differently
train_examples = [InputExample(texts=[sentence, sentence], label=0) for sentence in sentences]

# Create a HuggingFace Dataset from the sentence examples
data_dict = {'sentence_1': sentences, 'sentence_2': sentences}

# Correctly create a Dataset from the dictionary
dataset = Dataset.from_dict(data_dict)

# Create a DataLoader from the dataset
train_dataloader = DataLoader(train_examples, batch_size=2, shuffle=True)

# Load the pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Define the cosine similarity loss
train_loss = losses.CosineSimilarityLoss(model)

# Fine-tune the model
try:
    model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=4, warmup_steps=100)
    print("Model training completed successfully!")
except Exception as e:
    print(f"Error in model training: {e}")


Error in model training: name 'Dataset' is not defined


In [1]:
from datasets import Dataset

# Example data
data_dict = {
    "sentence_1": ["This is a sentence.", "Another sentence."],
    "sentence_2": ["This is a sentence.", "A different sentence."]
}

# Create a Dataset from the dictionary
dataset = Dataset.from_dict(data_dict)

# Print the dataset to verify
print(dataset)


Dataset({
    features: ['sentence_1', 'sentence_2'],
    num_rows: 2
})


In [5]:
from torch.utils.data import DataLoader
from sentence_transformers import InputExample, losses, SentenceTransformer
import torch

# Create a sample dataset with similarity scores as labels (1.0 for similar, 0.0 for dissimilar)
dataset = [
    {"sentence_1": "Hello, how are you?", "sentence_2": "Hi, how have you been?", "label": 1.0},
    {"sentence_1": "What is your name?", "sentence_2": "Could you tell me your name?", "label": 1.0},
    {"sentence_1": "I enjoy reading books.", "sentence_2": "Reading is my favorite hobby.", "label": 1.0},
    {"sentence_1": "It's a sunny day.", "sentence_2": "The weather is bright and sunny.", "label": 1.0},
]

# Prepare the data for training
train_examples = [
    InputExample(texts=[row["sentence_1"], row["sentence_2"]], label=row["label"])
    for row in dataset
]
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=2)

# Load a pre-trained SentenceTransformer model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Define a loss function
train_loss = losses.CosineSimilarityLoss(model)

# Fine-tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=4, warmup_steps=100)

# Save the fine-tuned model
model.save('fine_tuned_sentence_transformer')
print("Model fine-tuning completed and saved!")


Step,Training Loss


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Model fine-tuning completed and saved!


In [7]:
import torch
from transformers import AutoTokenizer, AutoModel
import accelerate


In [11]:
import transformers
from accelerate import Accelerator

print("Transformers and Accelerate are working correctly!")


Transformers and Accelerate are working correctly!


In [7]:
# Load the fine-tuned model
fine_tuned_model = SentenceTransformer('fine_tuned_sentence_transformer')

# Example usage: Compute embeddings for sentences
sentences = ["I love programming.", "Coding is my passion."]
sentence_embeddings = fine_tuned_model.encode(sentences)

# Output the embeddings
print(sentence_embeddings)


[[-1.96326468e-02 -1.79807190e-02  1.01756072e-02 -1.51771549e-02
   5.54744001e-05 -6.93052411e-02  6.68288991e-02  6.26461133e-02
   1.88734625e-02  4.84342650e-02 -8.44402835e-02  4.35467288e-02
   3.02871671e-02  7.58361816e-03  3.31148952e-02 -2.44786330e-02
  -7.93212503e-02 -5.11649549e-02  3.63172847e-03 -6.62944019e-02
  -1.32014826e-01 -7.16472231e-03 -2.76506413e-02 -3.44766043e-02
   6.28687441e-02  1.01452537e-01 -5.73814800e-03 -5.20127006e-02
   1.42729154e-03 -7.09168389e-02 -1.00487813e-01  1.16416015e-01
   6.41701743e-02  5.97182438e-02  1.41863273e-02  5.57095259e-02
   8.12012777e-02 -6.41406551e-02  1.54892961e-03  7.68235186e-03
  -9.65531543e-02  5.58917597e-02  4.01154608e-02 -2.50361189e-02
  -9.02409479e-03 -5.09955026e-02 -2.76898891e-02 -5.12793213e-02
   7.57109523e-02  2.26003360e-02  6.78671245e-03 -1.23196011e-02
   5.08785961e-05 -3.65584940e-02 -1.36210979e-03 -1.11579942e-02
   7.82819390e-02  1.12473201e-02 -2.08325237e-02 -3.30178775e-02
  -5.19911

In [9]:
import pickle

# Save the embeddings to a file
with open("sentence_embeddings.pkl", "wb") as f:
    pickle.dump(sentence_embeddings, f)

print("Embeddings saved successfully!")


Embeddings saved successfully!


In [11]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Query sentence
query = "I enjoy coding."

# Encode the query using the fine-tuned model
query_embedding = fine_tuned_model.encode([query])

# Calculate cosine similarity between the query and stored embeddings
similarities = cosine_similarity(query_embedding, sentence_embeddings)

# Get the index of the most similar sentence
most_similar_index = np.argmax(similarities)

# Print the most similar sentence
print(f"Most similar sentence: {dataset[most_similar_index]['sentence_2']}")


Most similar sentence: Could you tell me your name?


In [13]:
import streamlit as st

# Streamlit app
st.title("Sentence Similarity Search")

query = st.text_input("Enter a sentence:")

if query:
    query_embedding = fine_tuned_model.encode([query])
    similarities = cosine_similarity(query_embedding, sentence_embeddings)
    most_similar_index = np.argmax(similarities)
    
    st.write(f"Most similar sentence: {dataset[most_similar_index]['sentence_2']}")


2024-11-20 22:45:56.761 
  command:

    streamlit run E:\Anaconda\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [1]:
# from sklearn.cluster import KMeans

# # Set n_jobs to 1 to avoid multithreading issues
# kmeans = KMeans(n_jobs=1)  # Ensure that the KMeans uses a single thread for processing

# # Perform clustering and assign the labels
# labels = kmeans.fit_predict(embeddings)
